In [1]:
%load_ext autoreload
%autoreload 2

from numpy.random import seed
seed(123)

In [2]:
from scripts.audio_feature_extraction_for_file import extract_audio_features_for_frames
from scripts.utils import create_and_save_lookup_table
from scripts.biosignal_data_extraction import get_biosignal_data_for_frames
from scripts.video_landmark_extraction import get_mediapipe_pose_estimation_for_frames, get_dlib_face_estimation_for_frames
from scripts.csv_out import write_features_to_csv
from scripts.process_data import process_data_folder

2024-06-19 09:13:39.962134: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
# Save the lookup table to a CSV file -- only run once to get the file if it doesn't already exist 
# create_and_save_lookup_table('frequency_to_note.csv')

In [35]:
# Test that feature extraction works as expected on test audio
TEST_FILE = 'media/test_audio.wav'
features, cleaned_time = extract_audio_features_for_frames(TEST_FILE, reference_audio=True, frame_duration_ms=30)

9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 414ms/step


In [21]:
# Ensure all feature lists have the same length
lengths = [len(feature) for feature in features.values()]
if len(set(lengths)) != 1:
    for feature in features.values():
        print(len(feature))
    raise ValueError("All feature lists must have the same length")

audio_features_length = lengths[0]
print(audio_features_length)

86


In [22]:
# testing getting mediapipe landmark pose per 10ms frame
TEST_VIDEO_FILE_PATH = 'media/test_phone_video.mp4'
TEST_OUTPUT_DIR_PATH = 'media/output/'

pose_landmarks = get_mediapipe_pose_estimation_for_frames(cleaned_time, TEST_VIDEO_FILE_PATH, TEST_OUTPUT_DIR_PATH)

INFO:root:Processing mediapipe pose estimation for media/test_phone_video.mp4.
I0000 00:00:1718815179.075748  134257 gl_context.cc:357] GL version: 2.1 (2.1 ATI-4.14.1), renderer: AMD Radeon Pro 570 OpenGL Engine
Processing Frames:   0%|          | 0/67 [00:00<?, ?it/s]W0000 00:00:1718815179.380285  185301 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1718815179.402179  185301 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/Users/Suvi/miniforge3/envs/data_extract_env/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
Processing Frames:

Ignoring empty video frame.


In [23]:
if len(pose_landmarks) != audio_features_length:
    raise ValueError("Face landmarks and audio features must have the same length")

In [24]:
face_landmarks = get_dlib_face_estimation_for_frames(cleaned_time, TEST_VIDEO_FILE_PATH, TEST_OUTPUT_DIR_PATH)

if len(face_landmarks) != audio_features_length:
    raise ValueError("Face landmarks and audio features must have the same length")

INFO:root:Processing dlib face landmarks for media/test_phone_video.mp4.
Processing Frames: 100%|██████████| 67/67 [00:13<00:00,  5.04it/s]


In [33]:
#test biosignal extraction 
TEST_BIO_FILE_PATH = 'media/test_biosignal_data.json'

biosignal_data = get_biosignal_data_for_frames(cleaned_time, TEST_BIO_FILE_PATH, frame_duration_ms=30)

for channel in biosignal_data:
    if len(biosignal_data[channel]) != audio_features_length:
        print(len(biosignal_data[channel]))
        raise ValueError("Biosignal data and audio features must have the same length")


In [62]:
# test:
file_info = {
    'participant_number' : "001",
    'age' : 24,
    'experience_level' : "Intermediate",
    'phonation' : "neutral",
    'recording_condition' : "normal",
    'phrase' : "simple triad",
    'clip_number' : 1
}

# TEST csv output for test file 
TEST_CSV = "data/output_test.csv"

write_features_to_csv(TEST_CSV, file_info, features, pose_landmarks, face_landmarks, biosignal_data)

In [ ]:
# test feature extraction from file system

DIR_IN = '/Volumes/music suvi/MASTERS THESIS UPF/VOICE_DATA_THESIS'
CSV_TEST = 'data/real_out_test.csv'
LOG_TEST = 'data/processed_folders.log'

process_data_folder(DIR_IN, CSV_TEST, LOG_TEST, num_folders_to_process=, frame_duration_ms=30)


In [ ]:
# FINAL RUNNING PROCESSING OF ALL DATA

DIR_IN = '/Volumes/music suvi/MASTERS THESIS UPF/VOICE_DATA_THESIS'
CSV_OUT = "/Volumes/music suvi/MASTERS THESIS UPF/VOICE_DATA_THESIS/voice_data_thesis.csv"
LOG_FILE = "/Volumes/music suvi/MASTERS THESIS UPF/VOICE_DATA_THESIS/processed_folders.log"

process_data_folder(DIR_IN, CSV_OUT, LOG_FILE, num_folders_to_process=0, frame_duration_ms=30)
